In [89]:
import pandas as pd

In [90]:
#Global Variables

# File names
## Import
csvFile = 'vgsales-12-4-2019.csv'

## Export
excelFileName = 'ficheiro.xlsx'
tab1Name = 'Raw Data'
tab2Name = 'Platform Rank'

# Pandas CSV configuration
encodingName = 'utf8'
separator = ','

# Pandas XLS configuration
engineName = 'xlsxwriter'


In [91]:
# Essa função é reponsável pelo load de um ficheiro CSV 
def extract_base(inCSVName):
    load_file = pd.read_csv(inCSVName,sep=separator,encoding=encodingName)
    return load_file
        

In [92]:
# Essa função é responsável por remover a primeira coluna de index do ficheiro
# o parâmetro axis=1 indica que o drop é na coluna.
def transform_remove_index(inFile):
    return inFile.drop(inFile.columns[0], axis=1)

In [93]:
# Essa função é responsável por criar uma porcentagem de linhas null
def count_null(inFile,inColumn):
    totalRows = len(inFile)
    countNull = 0
    for i, row in inFile.iterrows():
        if pd.isnull(row[inColumn]):
            countNull += 1
    porcent = round((countNull * 100) / totalRows,2)
    return porcent


In [94]:
# Função responsável por fazer um group by na coluna Platform e somar os valores de Global_Sales
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados
def soma_global_sales_platform(inFile):
    file = inFile.groupby('Platform')['Global_Sales'].sum().reset_index()
    file = file.sort_values(by='Global_Sales', ascending = False).reset_index()
    return file

In [95]:
# Essa função recebe como parâmetro o pd.ExcelWriter, o file e a tab do excel e escreve no ficheiro
def write_xls(inWriter,inFile,inTab):
    inFile.to_excel(inWriter, sheet_name=inTab,index=False)
    

In [96]:
writer = pd.ExcelWriter(excelFileName, engine=engineName)

file = extract_base(csvFile)

percentNull = count_null(file,'ESRB_Rating')

file = transform_remove_index(file)
write_xls(writer,file,tab1Name)

maxGlobalPlatform = soma_global_sales_platform(file)
write_xls(writer,maxGlobalPlatform,tab2Name)

writer.close()


57.66
